In [146]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, make_scorer

In [2]:
train = pd.read_csv('/Users/Shikhar/data/quora/train.csv')

In [3]:
test = pd.read_csv('/Users/Shikhar/data/quora/test.csv')

In [18]:
train.shape

(404287, 6)

In [19]:
test.shape

(2345790, 3)

In [11]:
train.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [12]:
test.isnull().sum()

test_id      0
question1    0
question2    0
dtype: int64

## Removing missing values

In [13]:
train = train.dropna(how='any',axis=0) 

In [14]:
test = test.dropna(how='any',axis=0) 

## Tokenizing questions

In [290]:
veczr = CountVectorizer(ngram_range=(1,2),tokenizer=tokenize)

In [291]:
voc = veczr.fit(pd.concat([train['question1'],train['question2']]))

In [292]:
trn_q1 = voc.transform(train['question1'])

In [293]:
trn_q2 = voc.transform(train['question2'])

### Dot product for similarity

In [294]:
dot_product = trn_q1.sign().multiply(trn_q2.sign())

In [295]:
dot_product = np.array(dot_product.sum(axis=1))

In [296]:
dot_product = pd.Series(dot_product.ravel())

### Length of each question

In [297]:
q1_length = np.array(trn_q1.sign().sum(axis=1))

In [298]:
q1_length = pd.Series(q1_length.ravel())

In [299]:
q2_length = np.array(trn_q2.sign().sum(axis=1))

In [300]:
q2_length = pd.Series(q2_length.ravel())

### Number of common words combined

In [301]:
word_count = trn_q1.sign()+ trn_q2.sign()

In [302]:
word_count = word_count.sign()

In [303]:
word_count = np.array(word_count.sum(axis=1))

In [304]:
word_count = pd.Series(word_count.ravel())

In [305]:
trn_final = pd.concat([train.is_duplicate,dot_product,q1_length,q2_length,word_count],axis=1)

In [306]:
trn_final = trn_final.rename(columns={0:'dot_product',1:'q1_length',2:'q2_length',3:'word_count'})

### Creating ratio of common by total words

In [307]:
trn_final['ratio_common'] = trn_final['dot_product']/trn_final['word_count']

In [308]:
trn_final = trn_final.dropna(how='any',axis=0) 

### Train_Valid split

In [318]:
x_train,x_val,y_train, y_val= train_test_split(trn_final['ratio_common'],trn_final['is_duplicate'], test_size = 0.2, random_state =42)

### Logistic regression

In [322]:
mlr = LogisticRegression(C = 0.1, dual = True, n_jobs = -1)
mlr.fit(pd.DataFrame(x_train),y_train)

/Users/Shikhar/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [325]:
pred_val = mlr.predict(pd.DataFrame(x_val))
pred_prob_val = mlr.predict_proba(pd.DataFrame(x_val))

# print("Accuracy of training:",(pred_train.T == y_train).mean())
# print("log-loss of training is:", log_loss(y_train,pred_prob_train))
print("Accuracy of validation:",(pred_val.T == y_val).mean())
print("log-loss of validation is:", log_loss(y_val,pred_prob_val))

Accuracy of validation: 0.630038215615
log-loss of validation is: 0.655669114886


In [324]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [313]:
m = RandomForestClassifier(n_estimators=10)

In [314]:
m.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [315]:
pred_val = m.predict(x_val)
pred_prob_val = m.predict_proba(x_val)

In [316]:
print("Accuracy of validation:",(pred_val.T == y_val).mean())
print("log-loss of validation is:", log_loss(y_val,pred_prob_val))

Accuracy of validation: 0.627119482543
log-loss of validation is: 0.691594939515


In [317]:
m.feature_importances_

array([ 0.24716,  0.75284])